In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#############################################
######### LOAD SUITE2P AND BINARIZE #########
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# input calcium traces .csv file from Inscopix
fname = '/home/cat/data/nathalie/2021-10-22-15-41-49_video_sched_0.csv'

c = calcium.Calcium()
c.verbose = True             # outputs additional information during processing
c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = os.path.split(fname)[0]

# Load inscopix data
c.save_python = True         # save output as .npz file
c.save_matlab = True         # save output as .mat file
c.fname_inscopix = fname
c.load_inscopix()


c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = os.path.split(fname)[0]

# set flags to save matlab and python data
c.save_python = True         # save output as .npz file 
c.save_matlab = True         # save output as .mat file

###############################################
##### PARAMETERS FOR RUNNING BINARIZATION #####
###############################################
c.min_width_event_onphase = c.sample_rate//2 # set minimum withd of an onphase event; default: 0.5 seconds
c.min_width_event_upphase = c.sample_rate//4 # set minimum width of upphase event; default: 0.25 seconds

############# PARAMTERS TO TWEAK ##############
#     1. Cutoff for calling somthing a spike:
#        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
#                                        higher -> less events; lower -> more events
#                                        start at default and increase if data is very noisy and getting too many noise-events
#c.min_thresh_std_onphase = 2.5      # set the minimum thrshold for onphase detection; defatul 2.5
#c.min_thresh_std_upphase = 2.5      # set the minimum thershold for uppohase detection; default: 2.5

#     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
#        This is stored in high_cutoff: default 0.5 to 1.0
#        The lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces (but we loose some temporal precision)
c.high_cutoff = 0.5              

#     3. Removing bleaching and drift artifacts using polynomial fits
#        This is stored in detrend_model_order
c.detrend_model_order = 1 # 1-3 polynomial fit
c.detrend_model_type = 'mode' # 'mode', 'polynomial'

#
c.mode_window = None #*30 # 120 seconds @ 30Hz 

################################################
########### RUN BINARIZATION STEP ##############
################################################
# 
# double check that we have set the STD thrshold at a reasonable level to catch biggest/highest SNR bursts
c.show_plots = True

#
c.sample_rate = 20
c.min_width_event_onphase = c.sample_rate   # the onphase needs to be at least 
c.min_width_event_upphase = c.sample_rate//3 # the upphase needs to be at least 1/3 of a second
c.recompute_binarization = True

# not used currently
c.dff_min = 0.02   #this is the final threshold required by putative spikes
c.percentile_threshold = 0.999999
c.use_upphase = True

#
print ("fluorescence shapes: ", c.F.shape)
#c.F = c.F[:,:1300*30]
c.show_plots =False
c.remove_ends = False                     # delete the first and last x seconds in case [ca] imaging had issues
c.detrend_filter_threshold = 0.001
c.mode_window = 30*30  # None: compute mode on entire time; Value: sliding window based - baseline detection # of frames to use to compute mode

c.binarize_fluorescence()


# generate standard randomized plots:
c.save_sample_traces()

#
save_image = True
c.show_rasters(save_image)


cells:  (480, 36026)
F times:  (36026,)
fluorescence shapes:  (480, 36026)

  Binarization parameters: 
        low pass filter low cuttoff:  0.5 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_onphase:  1.5
        min_thresh_std_upphase:  1.5
        min_width_event_onphase:  30
        min_width_event_upphase:  10
        min_width_event_oasis:  2
        min_event_amplitude:  1


100%|██████████| 480/480 [00:02<00:00, 207.36it/s]
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 480/480 [00:00<00:00, 1058.44it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 480/480 [00:00<00:00, 726.56it/s]


...saving data...


In [6]:
# #####################################################
# ############## LOAD INSCOPIX AND BINARIZE ###########
# #####################################################

# ######## OLD VERSION (PRE-MODE FILTERING) ##########



# # input calcium traces .csv file from Inscopix
# fname = '/media/cat/4TB/donato/laurenz/DON-005099/20210609/suite2p/plane0/DON-005099_20210609_TRACES_ACQ.csv'

# c = calcium.Calcium()
# c.verbose = True             # outputs additional information during processing
# c.recompute = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
# c.data_dir = os.path.split(fname)[0]

# # Load inscopix data
# c.save_python = True         # save output as .npz file
# c.save_matlab = True         # save output as .mat file
# c.fname_inscopix = fname
# c.load_inscopix()

# ###############################################
# ########## 3 parameters to tweak ##############
# ###############################################
# #     1. Cutoff for calling somthing a spike:
# #        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
# #                                        higher -> less events; lower -> more events
# #                                        start at default and increase if data is very noisy and getting too many noise-events
# c.min_thresh_std_Fluorescence_onphase = 2.5
# c.min_thresh_std_Fluorescence_upphase = 2.5

# #     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
# #        This is stored in high_cutoff: default 1.0
# #                             - the lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces
# c.high_cutoff = .5

# #     3. Removing bleaching and drift artifacts using polynomial fits
# #        This is stored in detrend_model_order
# c.detrend_model_order = 1 # 1-5 polynomial fit

# # 
# #c.F=c.F[250:260]
# c.binarize_fluorescence()

In [12]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
#fname = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/nathalie/plane0/binarized_traces.npz'
#fname = '/media/cat/4TB/donato/renan/renan/binarized_traces.npz'
# 
fname = '/home/cat/data/nathalie/binarized_traces.npz'

c = calcium.Calcium()
c.fname = fname
c.data_dir = os.path.split(fname)[0]
c.load_binarization()

# pick a random cell to visuzlie
cell_id = 220

# 
scale = 100

c.plot_cell_binarization(cell_id, scale)


In [12]:
######################################################
############# SHOW RASTERS ###########################
######################################################

fname = '/media/cat/4TB/donato/steffen/DON-004366/20210228/binarized_traces.npz'

c = calcium.Calcium()
c.fname = fname
c.load_binarization()


c.show_rasters()

(1027, 92900)
